## Hartree-Fock using PySCF

In the lectures we have covered the Hartree-Fock method of finding an approximation to the ground-state energy of the electrons in a molecule.  While it is theoretically possible to solve the relevant equations by hand, this is exceptionally long and tedious.  Instead, we typically use one of the many quantum chemistry packages which have been developed for the purpose of performing this procedure on a computer.  In this problem, we will use the pyscf package for this purpose.  First, we install the package (probably best to do this in a fresh environment):

In [1]:
import sys
!{sys.executable} -m pip install pyscf
import pyscf

Defaulting to user installation because normal site-packages is not writeable


We first need to tell PySCF what molecule we're interested in.  See if you can figure out what the two lines below mean:

In [2]:
geometry = 'H 0. 0. 0.; H 0. 0. 0.714'
basis = 'STO-3G'

Now we'll create the PySCF object which represents the molecule (don't ask us why the parameter is called "atom" instead of "molecule).

In [3]:
molecule = pyscf.gto.M(atom = geometry, basis = basis)

This is all we'll need to be able to run a simple Hartree-Fock calculation.  Of course, there are many options to tweak how the process will run, but we'll ignore them for now.  To run Hartree-Fock, we create an scf.RHF object and then use the kernel method to get the energy (in Hartrees):

In [4]:
hartree_fock = pyscf.scf.RHF(molecule)
hf_energy = hartree_fock.kernel()
print(hf_energy)

converged SCF energy = -1.11750269105473
-1.11750269105473


## Full configuration interaction with PySCF

As we discussed in the lectures, Hartree-Fock is only one of many computational methods in chemistry to estimate the ground state energy of the electrons in a molecule.  There is typically a trade-off between the accuracy of the method, and the computational difficulty (though perhaps not the conceptual difficulty!).  

In later lectures, we'll cover the Full Configuration Interaction method.  This is at the extreme end of the difficulty vs. accuracy method -- it is in fact an exact diagonalization of the electronic Hamiltonian.  As such, it is numerically exact, but exponentially hard (if it were not exponentially hard, we wouldn't need quantum computers and we'd be out of a job).  Due to the computational difficulty, it is not used much in earnest, but for small systems can be used as a benchmark for testing how accurate a computational method is.

As we'll come on to in later lectures, FCI requires a *reference state* to start from.  This is an approximate solution to the ground state.  In this instance, we can use our Hartree-Fock solution:

In [7]:
from pyscf import fci
full_configuration_interaction = pyscf.fci.FCI(hartree_fock)

Now we can just tell PySCF to run the FCI algorithm:

In [8]:
full_configuration_interaction.run()
fci_energy = full_configuration_interaction.kernel()[0]
print(fci_energy)

-1.1369181758408105


We know that the FCI energy is exact.  So if we compare our Hartree-Fock energy against our FCI energy, we obtain the error caused by the approximations made in the Hartree-Fock procedure.  

In [9]:
print(fci_energy-hf_energy)

-0.01941548478608035


Note how the FCI energy is lower than the HF energy, in agreement with the variational principle.  Also note that the error is actually quite small - it is common in chemistry for the error incurred by approximation to be maybe ~1% of the full energy, although this is very system dependent.  The danger is that these small errors can easily become large errors when propagated through a successor calculation.